# FAI P21 Sequence Post Processing

First read the ardupilot dataflash log file. The FlightData and ArdupilotLogReader packages can be found here: https://github.com/PyFlightCoach 
ArdupilotLogReader depends on pymavlink.

The log 00000100.BIN was from (by my standards) a very good flight, so its a good place to start to recognise the sequence.

In [1]:
from flightdata import Flight, Fields, CIDTypes

bin=Flight.from_log('./logs/00000100.BIN')

The Fields class contains information on the data contained within the flight class.

Here is some information extracted from the Fields object:

In [2]:
field_summary = [[field.name, str(field.names[:3]), str(field.unit), str(field.description)] for field in Fields.all()]

for row in field_summary:
    print("{:>15} {: >60} {: >20} {: >60}".format(*row))

           time                               ['time_flight', 'time_actual']               second                                                             
    tx_controls          ['tx_controls_0', 'tx_controls_1', 'tx_controls_2']               second                                PWM Values coming from the TX
         servos                         ['servos_0', 'servos_1', 'servos_2']               second                               PWN Values going to the Servos
           mode                               ['mode_0', 'mode_1', 'mode_2']                    1                                    The active flight mode ID
       position                   ['position_x', 'position_y', 'position_z']                meter                                  position of plane (n, e, d)
global_position    ['global_position_latitude', 'global_position_longitude']               degree                                                             
  gps_sat_count                               

The Flight class holds the data in SI units and provides some handy access methods. For example .subset returns a new Flight class for the subset, .read_fields returns a dataframe with the columnns matching the fields you requested. .read_field_tuples does the same thing then converts it to a tuple of numpy arrays.

Here I extract the position and attitude data between 100 and 101 seconds:

In [3]:
bin.subset(100, 101).read_fields([Fields.POSITION, Fields.ATTITUDE])

,position_x,position_y,position_z,attitude_roll,attitude_pitch,attitude_yaw
time_index,,,,,,
0.000000,-241.850739,-50.183926,-97.288467,-0.012043,0.652055,4.049338
0.098988,-243.044159,-51.832420,-98.644600,-0.008378,0.673348,4.049338
0.199839,-244.792816,-54.246998,-100.729065,-0.004712,0.707556,4.049164
0.298964,-245.938339,-55.831230,-102.173164,-0.001571,0.731118,4.048466
0.399976,-247.586716,-58.111042,-104.368965,0.000175,0.767945,4.046371
0.498957,-248.649048,-59.583282,-105.870430,0.000175,0.792903,4.041833
0.599959,-250.190231,-61.717567,-108.174126,-0.006283,0.833046,4.031362
0.698981,-251.178345,-63.084167,-109.744133,-0.012043,0.859051,4.020890
0.799938,-252.616089,-65.056183,-112.141098,-0.022340,0.898146,4.001516


I like to use plotly for figures. Information on how to set it up for JupyterLab is here: https://plotly.com/python/getting-started/#jupyterlab-support-python-35

Below are a few functions to hold some boilerplate associated with making graphs 

In [4]:
import plotly.graph_objects as go

def plot2d(datax, datay, colour='black', width=1, fig=None, name=None):
    if not fig:
        fig=go.Figure()
    fig.add_trace(go.Scatter(x=datax, y=datay, name=name, line=dict(color=colour, width=width)))
    fig.update_layout(width = 800, height = 500, margin=dict(l=20, r=20, t=20, b=20),)
    fig.update_yaxes(scaleanchor = "x",scaleratio = 1)
    return fig

def plot3d(datax, datay, dataz, colour='black',fig=None, width=1):
    if not fig:
        fig=go.Figure()
    fig.add_trace(go.Scatter3d(x=datax, y=datay, z=dataz, line=dict(color=colour, width=width, showscale=True),
        mode = 'lines'))
    fig.update_layout(width = 800, height = 500, margin=dict(l=0, r=0, t=0, b=0))


    fig.update_yaxes(scaleanchor = "x",scaleratio = 1)
    return fig

plot3d(*bin.read_field_tuples(Fields.POSITION)).show()

The flight is upside-down in the plot above. The default coordinate system for the position and attitude data is North (x), East (y), Down (z). It would also be handy if the data were rotated so the flight aligns with a principal axis. 

The geometry package, also found here: https://github.com/PyFlightCoach, gives some handy classes for dealing with 3D geometry. The Point class handles points, the Coord class handles coordinate frames. 

The rotate data function below returns a new Flight object containing the position, attitude and velocity data rotated about the z axis.   

In [5]:
from geometry import Point, Coord
from math import pi
import numpy as np
import pandas as pd 


def rotate_data(bin, angle):
    # get a vector pointing in the desired x axis by rotating the default x axis vector by the angle requested.
    box_normal_vector = Point(1, 0, 0).rotate(Point(0,0,angle).to_rotation_matrix())

    #create a new Coord based on the origin (unchanged), the z axis (up), and the new x axis vector. 
    box_cid = Coord.from_zx(Point(0, 0, 0), Point(0, 0, -1), box_normal_vector)

    def point_to_box(x, y, z):
        box_point = Point(x, y, z).rotate(box_cid.rotation_matrix)
        return box_point.x, box_point.y, box_point.z

    def euler_to_box(x, y, z):
        seupoint = Point(x, y - pi, - z - angle)
        return seupoint.x, seupoint.y, seupoint.z

    transforms = {i: lambda *args: args  for i in range(0, 7)}
    transforms[CIDTypes.CARTESIAN] = point_to_box
    transforms[CIDTypes.EULER] = euler_to_box
    transforms[CIDTypes.ZONLY] = lambda *args: tuple(-arg for arg in args) 
    transforms[CIDTypes.XY]: lambda *args: point_to_box(*args, z=0) 
    
    return bin.transform(transforms)


flight = rotate_data(bin, -126*pi/180)

plot2d(flight.data.position_x, flight.data.position_y).show()
plot2d(flight.data.position_x, flight.data.position_z).show()
plot3d(*flight.read_field_tuples(Fields.POSITION)).show()


In [6]:
fig = plot3d(
    *flight.read_field_tuples(Fields.POSITION), 
    colour=np.vectorize(lambda *args: abs(Point(*args)))(*flight.read_field_tuples(Fields.VELOCITY)), 
    width=4)
fig.update_yaxes(scaleanchor = "x",scaleratio = 1)
fig.show()